# **MAIN RUN LOCAL_PIPELINE**

(NB: Penjelasan tahap-tahap ada di bawah section ini)

In [1]:
!pip show tfx

Name: tfx
Version: 1.14.0
Summary: TensorFlow Extended (TFX) is a TensorFlow-based general-purpose machine learning platform implemented at Google.
Home-page: https://www.tensorflow.org/tfx
Author: Google LLC
Author-email: tensorflow-extended-dev@googlegroups.com
License: Apache 2.0
Location: /mnt/B28A5BE18A5BA0A1/DataKu/Downloads/ALTER_CODING/MLOPS_CLOUDEKA/ubuntu/env2/lib/python3.9/site-packages
Requires: absl-py, apache-beam, attrs, click, docker, google-api-core, google-api-python-client, google-apitools, google-cloud-aiplatform, google-cloud-bigquery, grpcio, jinja2, keras-tuner, kubernetes, ml-metadata, ml-pipelines-sdk, numpy, packaging, portpicker, protobuf, pyarrow, pyyaml, tensorflow, tensorflow-data-validation, tensorflow-hub, tensorflow-model-analysis, tensorflow-serving-api, tensorflow-transform, tfx-bsl, typing-extensions
Required-by: 


In [5]:
!tree

.
├── config
│   └── prometheus.config
├── dataset
│   └── clickbait.csv
├── Dockerfile
├── main_runlocal_pipeline_clickbait.ipynb
├── local_pipeline.py
├── modules
│   ├── components.py
│   ├── trainer.py
│   ├── transform.py
│   └── tuner.py
├── monitoring
│   ├── Dockerfile
│   └── prometheus.yml
├── Pylint_check.ipynb
└── requirements.txt

4 directories, 13 files


In [9]:
import time
start_time = time.time()

!python local_pipeline.py

print("--- %s seconds ---" % (time.time() - start_time))

Streaming output truncated to the last 5000 lines.
INFO:absl:Node Evaluator depends on ['Run[CsvExampleGen]', 'Run[Latest_blessed_model_resolver]', 'Run[Trainer]'].
INFO:absl:Node Evaluator is scheduled.
INFO:absl:Node Pusher depends on ['Run[Evaluator]', 'Run[Trainer]'].
INFO:absl:Node Pusher is scheduled.
INFO:absl:node Latest_blessed_model_resolver is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.dsl.components.common.resolver.Resolver"
  }
  id: "Latest_blessed_model_resolver"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "sitomb-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20240329-084051.264027"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "sitomb-pipeline.Latest_blessed_model_resolver"
      }
    }
  }
}
inputs

In [44]:
!tree

.
├── config
│   └── prometheus.config
├── dataset
│   └── clickbait.csv
├── Dockerfile
├── local_pipeline.py
├── main_run_local_pipeline_clickbait.ipynb
├── modules
│   ├── components.py
│   ├── trainer.py
│   ├── transform.py
│   └── tuner.py
├── monitoring
│   ├── Dockerfile
│   └── prometheus.yml
├── output
│   ├── serving_model
│   │   └── 1711701816
│   │       ├── assets
│   │       ├── fingerprint.pb
│   │       ├── keras_metadata.pb
│   │       ├── saved_model.pb
│   │       └── variables
│   │           ├── variables.data-00000-of-00001
│   │           └── variables.index
│   └── sitomb-pipeline
│       ├── CsvExampleGen
│       │   └── examples
│       │       └── 2
│       │           ├── Split-eval
│       │           │   └── data_tfrecord-00000-of-00001.gz
│       │           └── Split-train
│       │               └── data_tfrecord-00000-of-00001.gz
│       ├── Evaluator
│       │   ├── blessing
│       │   │   └── 36
│       │   │       └── BLESSED
│       │   └── evalu

# **PENJELASAN DARI TAHAP-TAHAP**

## **1. FILE local_pipeline.py**

File `local_pipeline.py` adalah sebuah modul Python yang bertanggung jawab untuk menginisialisasi dan menjalankan sebuah pipeline machine learning menggunakan TensorFlow Extended (TFX) dalam lingkungan lokal. Modul ini berfungsi sebagai titik masuk utama atau skrip pengatur yang mengatur alur kerja keseluruhan dari pipeline ML, mulai dari pra-pemrosesan data hingga pelatihan model dan penyimpanan hasilnya.

### Tugas Utama:
1. **Inisialisasi Komponen Pipeline**: Memuat dan mengonfigurasi komponen-komponen TFX seperti `CsvExampleGen`, `StatisticsGen`, `SchemaGen`, `ExampleValidator`, `Transform`, `Tuner`, `Trainer`, `Resolver`, `Evaluator`, dan `Pusher`.
2. **Penentuan Konfigurasi Pipeline**: Menetapkan parameter-parameter seperti lokasi data masukan, lokasi output, dan konfigurasi lain yang diperlukan untuk menjalankan pipeline dengan benar.
3. **Pembentukan Pipeline**: Menggabungkan semua komponen dan konfigurasi ke dalam sebuah objek `pipeline.Pipeline` yang mewakili alur eksekusi dari setiap komponen dalam pipeline.
4. **Eksekusi Pipeline**: Menjalankan pipeline menggunakan `BeamDagRunner`, yang akan mengeksekusi setiap komponen secara berurutan sesuai dengan alur yang ditentukan.

Dengan menggunakan file `local_pipeline.py`, pengguna dapat dengan mudah mengatur dan menjalankan pipeline ML mereka dalam lingkungan lokal dengan hanya melakukan pengaturan konfigurasi yang sesuai dan menjalankan skrip tersebut. Ini memberikan fleksibilitas dan kemudahan dalam pengembangan, pengujian, dan penyebaran model ML.

In [ ]:
# %%writefile "local_pipeline.py"

"""
Author: Tomb
Date: 3/30/2024
This is the local_pipeline.py module.
Usage:
    this is the main of starting pipeline for running
    pipelne. This module will create pipeline environtment
    and initialize it.
"""

import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

PIPELINE_NAME = "sitomb-pipeline"

# pipeline inputs
DATA_ROOT = "dataset"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TRAINER_MODULE_FILE = "modules/trainer.py"
TUNER_MODULE_FILE = "modules/tuner.py"

# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")


def init_local_pipeline(component_arg, pipeline_root_arg: Text
                        ) -> pipeline.Pipeline:
    """
    init_local_pipeline that start everything

    Init local pipeline for initialize pipeline
    and defined environtment above (Output_Base dkk)

    Returns:
    TFX pipeline sent to apache beam
    """

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root_arg,
        components=component_arg,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )


if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components

    components = init_components({
        "data_dir": DATA_ROOT,
        "training_module": TRAINER_MODULE_FILE,
        "transform_module": TRANSFORM_MODULE_FILE,
        "tuner_module": TUNER_MODULE_FILE,
        "training_steps": 5000,
        "eval_steps": 1000,
        "serving_model_dir": serving_model_dir
    })

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)


## **2. MODUL components.py**

## components.py

Modul `components.py` adalah bagian dari sistem pipeline Machine Learning yang bertanggung jawab untuk menginisialisasi komponen-komponen yang diperlukan dalam alur kerja pipeline. Modul ini mendefinisikan fungsi `init_components` yang menghasilkan dan mengkonfigurasi komponen-komponen TFX seperti `CsvExampleGen`, `StatisticsGen`, `SchemaGen`, `ExampleValidator`, `Transform`, `Tuner`, `Trainer`, `Resolver`, `Evaluator`, dan `Pusher`.

### Fungsi Utama:
- **init_components(arguments)**: Fungsi ini menerima sebuah argumen `arguments` berupa dictionary yang berisi konfigurasi seperti lokasi data, modul transformasi, modul pelatihan, jumlah langkah pelatihan, jumlah langkah evaluasi, dan direktori model yang akan disajikan. Fungsi ini mengembalikan tuple yang berisi semua komponen yang telah diinisialisasi dan dikonfigurasi sesuai dengan argumen yang diberikan.

Dengan menggunakan modul `components.py`, pengguna dapat dengan mudah mengatur dan mengkonfigurasi komponen-komponen dalam sistem pipeline mereka dengan hanya memberikan argumen yang sesuai ke dalam fungsi `init_components`. Ini memungkinkan pengguna untuk mempercepat pengembangan dan penyesuaian alur kerja pipeline mereka sesuai kebutuhan.


In [ ]:
!mkdir "modules"

In [ ]:
# %%writefile "modules/components.py"

"""
Author: Tomb
Date: 3/30/2024
This is the components.py module.
Usage:
    this is the main of starting pipeline
    components for running
    pipeline.
"""
import os
import tfx
import tensorflow_model_analysis as tfma
from tfx.components import (CsvExampleGen, StatisticsGen,
                    SchemaGen, ExampleValidator, Transform,
                    Tuner, Trainer, Evaluator, Pusher)
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy
)

LABEL_KEY = "clickbait"
FEATURE_KEY = "headline"

def init_components(arguments):
    """Initiate tfx pipeline components

    arguments is dictionary include:
        data_dir (str): a path to the data
        transform_module (str): a path to the transform_module
        training_module (str): a path to the transform_module
        training_steps (int): number of training steps
        eval_steps (int): number of eval steps
        serving_model_dir (str): a path to the serving model directory

    Returns:
        TFX components
    """

    example_gen = CsvExampleGen(
        input_base=arguments["data_dir"]
    )

    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"]
    )

    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"]
    )

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )

    transform = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=os.path.abspath(arguments["transform_module"])
    )

    tuner = Tuner(
        module_file=os.path.abspath(arguments["tuner_module"]),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        train_args=tfx.v1.proto.TrainArgs(
            splits=["train"],
            num_steps=arguments["training_steps"],
        ),
        eval_args=tfx.v1.proto.EvalArgs(
            splits=["eval"],
            num_steps=arguments["eval_steps"],
        ),
    )

    trainer = Trainer(
        module_file=os.path.abspath(arguments["training_module"]),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        hyperparameters=tuner.outputs["best_hyperparameters"],
        train_args=tfx.v1.proto.TrainArgs(
            splits=['train'],
            num_steps=arguments["training_steps"]),
        eval_args=tfx.v1.proto.EvalArgs(
            splits=['eval'],
            num_steps=arguments["eval_steps"])
    )

    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')

    slicing_specs = [
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=FEATURE_KEY)
    ]

    metrics_specs = [
        tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': 0.0001})
                        )
                )
            ])
    ]

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key=LABEL_KEY)],
        slicing_specs=slicing_specs,
        metrics_specs=metrics_specs
    )

    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config)

    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=tfx.v1.proto.PushDestination(
            filesystem=tfx.v1.proto.PushDestination.Filesystem(
                base_directory=arguments["serving_model_dir"]
            )
        ),
    )

    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )
    return components


## **3. MODUL transform.py**

Modul `transform.py` merupakan bagian dari pipeline Machine Learning yang bertanggung jawab untuk melakukan pra-pemrosesan atau transformasi data sebelum dilakukan pelatihan model. Modul ini menyediakan fungsi-fungsi untuk melakukan transformasi fitur-fitur input menjadi format yang sesuai untuk diproses oleh model.

### Fungsi Utama:
- **transformed_name(key)**: Fungsi ini mengembalikan nama fitur yang telah ditransformasi berdasarkan kunci fitur yang diberikan. Misalnya, fitur kunci "headline" dapat diubah menjadi "headline_xf" setelah transformasi.
- **preprocessing_fn(inputs)**: Fungsi ini melakukan pra-pemrosesan pada fitur-fitur input yang diberikan. Fitur-fitur input diubah sesuai dengan kebutuhan, seperti mengonversi teks menjadi huruf kecil atau mengubah label menjadi tipe data yang sesuai.

Dengan menggunakan modul `transform.py`, pengguna dapat dengan mudah melakukan pra-pemrosesan data sebelum dilakukan pelatihan model. Ini memungkinkan pengguna untuk mempersiapkan data dengan cara yang sesuai dengan kebutuhan model mereka sebelum proses pelatihan dimulai.


In [ ]:
"""
Author: Tomb
Date: 3/27/2024
This is the transform.py module.
Usage:
- For Transorm Feature into integer. Headline string > integer
"""

import tensorflow as tf

LABEL_KEY = "clickbait"
FEATURE_KEY = "headline"


def transformed_name(key):
    """Transform feature key

    Args:
        key (str): the key to be transformed

    Returns:
        str: transformed key
    """

    return f"{key}_xf"


def preprocessing_fn(inputs):
    """Preprocess input features into transformed features

    Args:
        inputs (dict): map from feature keys to raw features

    Returns:
        dict: map from features keys to transformed features

    """

    outputs = {}

    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(
        inputs[FEATURE_KEY])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs


## **4. MODUL Tuner.py**
Modul `tuner.py` adalah bagian dari pipeline Machine Learning yang menyediakan fungsionalitas untuk melakukan penyetelan otomatis terhadap hiperparameter model menggunakan KerasTuner. Modul ini bertanggung jawab untuk mencari hiperparameter terbaik untuk model ML melalui eksperimen berbagai kombinasi hiperparameter.

### Fungsi Utama:
- **tuner_fn(fn_args)**: Fungsi ini merupakan inti dari modul tuner yang digunakan dalam komponen `Tuner` pada pipeline ML. Fungsi ini menerima argumen `fn_args` yang berisi informasi tentang lokasi data, grafik transformasi, dan hiperparameter yang akan disetel. Selanjutnya, fungsi ini mengembalikan objek tuner beserta argumen-argumen untuk melatih model dengan hiperparameter terbaik yang ditemukan.

### Fitur Utama:
- **build_model(hp, vectorizer_layer)**: Fungsi ini digunakan untuk membangun model ML dengan arsitektur yang ditentukan. Fungsi ini menerima objek Hyperparameters (`hp`) dari KerasTuner dan layer vektorisasi teks sebagai argumen. Selanjutnya, fungsi ini mengembalikan model Keras yang telah dibangun dengan arsitektur yang sesuai dengan hiperparameter yang ditemukan.
- **input_data(file_pattern, transform_output, num_epochs, batch_size)**: Fungsi ini digunakan untuk mempersiapkan dataset untuk pelatihan dan evaluasi model. Fungsi ini menerima pola file, output transformasi, jumlah epoch, dan ukuran batch sebagai argumen, dan mengembalikan dataset yang siap untuk digunakan.

Modul `tuner.py` memungkinkan pengguna untuk secara otomatis menemukan kombinasi hiperparameter terbaik untuk model ML mereka dengan menggunakan teknik penyetelan otomatis. Ini membantu meningkatkan kinerja dan generalisasi model tanpa perlu melakukan penyetelan secara manual.


In [ ]:
"""
Author: Tomb
Date: 3/30/2024
This module provides auto hyperparameter tuning for TensorFlow
models in TFX pipelines using KerasTuner.
Usage:
    Use within a TFX pipeline component
    for optimizing model
    parameters based on specified
    metrics and trials.
"""
from typing import NamedTuple, Dict, Text, Any
from tfx.components.trainer.fn_args_utils import FnArgs
from kerastuner.engine import base_tuner
from kerastuner import Hyperband, Objective
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_transform as tft
from transform import (LABEL_KEY, FEATURE_KEY, transformed_name)

# Constants
NUM_EPOCHS = 1
MAX_TOKEN = 5000
SEQUENCE_LENGTH = 100

# NamedTuple for Tuner Function Result
TunerResult = NamedTuple("TunerResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

# Early stopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=5,
)


def gzip_reader(filenames):
    """
    Creates a TFRecordDataset to read TFRecord files with GZIP compression.
        This function takes a list of filenames pointing to
        TFRecord files compressed using GZIP and returns a `tf.data.TFRecordDataset`
        object configured to read them.

    Parameters:
    - filenames (list of str or str): A list of filenames (paths) of the
        TFRecord files to read. If a single filename is provided,
        it is wrapped in a list automatically.
    """
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_data(file_pattern, transform_output, num_epochs, batch_size=64):
    """
    Prepares a batched dataset from TFRecord files using the provided transformation schema.
     Parameters:
    - file_pattern (str): A string pattern that matches the TFRecord files to read.
    - transform_output: object containing the transformation schema 
    - num_epochs (int): The number of times the dataset should be repeated. 
    - batch_size (int): The number of records to combine in a single batch.
    """
    transform_feature_spec = (
        transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset

# Model Building Function


def build_model(hp, vectorizer_layer):
    """
    Create model based
    """
    num_hidden_layers = hp.Choice("num_hidden_layers", values=[1, 2, 3])
    embed_dims = hp.Int("embed_dims", min_value=16, max_value=256, step=32)
    lstm_units = hp.Int("lstm_units", min_value=32, max_value=256, step=32)
    dense_units = hp.Int("dense_units", min_value=64, max_value=512, step=64)
    dropout_rate = hp.Float(
        "dropout_rate",
        min_value=0.2,
        max_value=0.5,
        step=0.1)
    learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 1e-5])

    inputs = tf.keras.Input(
        shape=(
            1,
        ),
        name=transformed_name(FEATURE_KEY),
        dtype=tf.string)

    x = vectorizer_layer(inputs)
    x = layers.Embedding(input_dim=5000, output_dim=embed_dims)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

    for _ in range(num_hidden_layers):
        x = layers.Dense(dense_units, activation=tf.nn.relu)(x)
        x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["binary_accuracy"],
    )

    return model

# Tuner Function


def tuner_fn(fn_args: FnArgs):
    """
    main tuner function
    """
    transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_data = input_data(
        fn_args.train_files[0], transform_output, NUM_EPOCHS
    )
    eval_data = input_data(
        fn_args.eval_files[0], transform_output, NUM_EPOCHS
    )

    vectorizer_dataset = train_data.map(
        lambda f, l: f[transformed_name(FEATURE_KEY)]
    )

    vectorizer_layer = layers.TextVectorization(
        max_tokens=MAX_TOKEN,
        output_mode="int",
        output_sequence_length=SEQUENCE_LENGTH,
    )
    print("DEBUG: Start_vectorizer_adapt")
    vectorizer_layer.adapt(vectorizer_dataset)
    print("DEBUG: Finish Vectorizer Adapt")

    tuner = Hyperband(
        hypermodel=lambda hp: build_model(hp, vectorizer_layer),
        objective=Objective('val_binary_accuracy', direction='max'),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="clickbait_detection_hyperband_kt",
    )

    return TunerResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping_cb],
            "x": train_data,
            "validation_data": eval_data,
            "steps_per_epoch": 50,
            "validation_steps": 25,
        },
    )


## **5. MODUL trainer.py**
Modul `trainer.py` merupakan bagian dari pipeline Machine Learning yang bertanggung jawab untuk melatih model ML dengan menggunakan data yang telah diproses sebelumnya. Modul ini menyediakan fungsi-fungsi untuk membangun, melatih, dan mengevaluasi model ML menggunakan TensorFlow.

### Fungsi Utama:
- **model_builder(hyperparameters, show_summary=True)**: Fungsi ini digunakan untuk membangun arsitektur model ML berdasarkan hiperparameter yang diberikan. Fungsi ini menerima hiperparameter, seperti jumlah lapisan tersembunyi, ukuran batch, dan tingkat dropout, serta menampilkan ringkasan model jika `show_summary` diatur ke `True`.
- **run_fn(fn_args)**: Fungsi ini merupakan inti dari modul trainer yang digunakan dalam komponen `Trainer` pada pipeline ML. Fungsi ini menerima argumen `fn_args` yang berisi informasi tentang lokasi data latih, data evaluasi, grafik transformasi, dan hiperparameter yang akan digunakan. Selanjutnya, fungsi ini membangun, melatih, dan mengevaluasi model ML dengan menggunakan data yang diberikan.

### Fitur Utama:
- **input_fn(file_pattern, tf_transform_output, num_epochs, batch_size)**: Fungsi ini digunakan untuk mempersiapkan dataset untuk pelatihan dan evaluasi model. Fungsi ini menerima pola file, output transformasi, jumlah epoch, dan ukuran batch sebagai argumen, dan mengembalikan dataset yang siap digunakan.
- **_get_serve_tf_examples_fn(model, tf_transform_output)**: Fungsi ini digunakan untuk membuat fungsi TensorFlow untuk melayani contoh-contoh TensorFlow yang di-serialisasi secara mentah. Fungsi ini membantu dalam menyajikan model yang telah dilatih secara online menggunakan TensorFlow Serving.

Modul `trainer.py` memungkinkan pengguna untuk membangun, melatih, dan mengevaluasi model ML dengan mudah menggunakan TensorFlow. Ini memfasilitasi pengguna dalam menjalankan pelatihan model secara efisien dan melakukan evaluasi kinerja model untuk pengembangan yang lebih lanjut.


In [ ]:
"""
Author: Tomb
Date: 3/30/2024
This is the trainer.py module.
Usage:
    this is the main of training pipeline
    models when running pipeline.
"""
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from transform import (
    LABEL_KEY,
    FEATURE_KEY,
    transformed_name,
)

VOCAB_SIZE = 10000
EMBEDDING_DIM = 16
SEQUENCE_LENGTH = 100


def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
             tf_transform_output,
             num_epochs,
             batch_size=64) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset


vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


def model_builder(hyperparameters, show_summary=True):
    """
    This function defines a Keras model and returns the model as a
    Keras object.
    """
    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            tf.keras.Input(shape=(1,), name=transformed_name(feature))
        )

    concatenate = tf.keras.layers.concatenate(input_features)
    
    deep = tf.keras.layers.Dense(
        hyperparameters["dense_unit"], activation=tf.nn.relu)(concatenate)

    for _ in range(hyperparameters["num_hidden_layers"]):
        deep = tf.keras.layers.Dense(
            hyperparameters["dense_unit"], activation=tf.nn.relu)(deep)
        deep = tf.keras.layers.Dropout(hyperparameters["dropout_rate"])(deep)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    if show_summary:
        model.summary()

    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """
    Creates a TensorFlow serving function that processes raw serialized TF examples.
    """

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn


def run_fn(fn_args: FnArgs) -> None:
    """
    Main function of trainer module
    """

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        mode='max',
        verbose=1,
        patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor='val_binary_accuracy',
        mode='max',
        verbose=1,
        save_best_only=True)

    hyperparameters = fn_args.hyperparameters["values"]
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    print("vectorize_layer_start")
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
            for i in list(train_set)]])

    print("Build_model_start")
    model = model_builder(hyperparameters, show_summary=True)

    model.fit(x=train_set,
              validation_data=val_set,
              callbacks=[tensorboard_callback, es, mc],
              steps_per_epoch=1000,
              validation_steps=1000,
              epochs=10)
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(
            model,
            tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'))}
    model.save(
        fn_args.serving_model_dir,
        save_format='tf',
        signatures=signatures)
